# Graded Assignment 3 - Decision Trees

* Algorithm is built with CART, uses Gini index and Gini impurity to build the tree.


In [1]:
class DecisionTreeClassifier:
    def __init__(self, df):
        
        self.init_dict = {}
        self.sum_dict = {}
        self.df = df
        
        self.cols = self.df.columns.values.tolist()
        self.features = self.cols[1:]
        self.res = self.cols[-1]

        for col in self.features:
            if col != self.res:
                
                self.init_dict, self.sum_dict = self.built_dictionary(col,self.df)
                
    

    def built_dictionary(self, col, dataframe):
        """Builds a dictionary based by group by, returns:
        Init_dict: Count of results (yes/no) per feature and instance, 
                used by select_feature function to calculate gini
        Sum_dict: Sum of results per feature and instance, used by select_feature function to calculate gini """
        
        self.dataframe = dataframe
        
        self.init_dict.update({col: {k: f.groupby(self.res).size().to_dict()
                    for k, f in dataframe[[col,self.res]].groupby(col)}})

        self.sum_dict.update({col: dataframe[[col,self.res]].groupby(col).count().to_dict()})
        
        return self.init_dict, self.sum_dict
        
        
    def calculate_gini_feature(self, yes, no, total):
        """Calculates the gini index for each instance """
        
        return 1-pow((yes/total),2)-pow((no/total),2)


    def calculate_gini_weigted(self, gini_list_feature):
        """Calculates the weighted gini index for the feature """
        
        self.score = 0
        gr_total = sum(n for _, n in gini_list_feature)
        for i in range(len(gini_list_feature)):
            self.score = self.score + ((gini_list_feature[i][1]/gr_total)*gini_list_feature[i][0])   
        return self.score

    def select_feature(self):
        """ Finds the feature with lowest gini impurity from init_dict and sum_dict calculated above """
        
        self.gini_list= []

        self.init_key_list = list(self.init_dict.keys())

        self.gini_result_list = []

        for key in self.init_key_list:
            for key2 in self.init_dict[key].keys():
                try:
                    self.count_yes = self.init_dict[key][key2]['Yes']
                except:
                    self.count_yes = 0
                try:
                    self.count_no = self.init_dict[key][key2]['No']
                except:
                    self.count_no = 0

                self.count_total = self.sum_dict[key][self.res][key2]
                
                self.gini_feature = self.calculate_gini_feature(self.count_yes, self.count_no, self.count_total)
                self.gini_list.append((self.gini_feature, self.count_total))
            
            self.weighted_gini = self.calculate_gini_weigted(self.gini_list)
            self.gini_result_list.append((key,self.weighted_gini))
            

        self.gini_result_list.sort(key=lambda x:x[1])

        self.start_feature = self.gini_result_list[0][0]
        
        return self.start_feature
    
    def check_length(self,dictionary):
        """Checks the lenght of the dictionary as a stop criteria. 
        In case there is only 1 cluster in the dictionary (only yes or no), 
        positions the result to the end and closes the branch. """
        
        self.dictionary = dictionary
        
        self.key_leng = [len(dictionary[key]) for key in dictionary.keys()]

        if all(self.key_leng[x] == 1 for x in range(len(self.key_leng))):
            return True
        else:
            return False
        
        

    def build_tree(self, root):
        """Takes the root as input and builds the decision tree"""
        self.root = root
        
        self.final_dict = []

        self.features.remove(self.root)
        
        # Instances of the root node, 
        # data is filtered based on each instance to find childs that has only one cluster
        self.filters = list(self.init_dict[self.root].keys())
        
        self.init_dict = {}
        self.sum_dict = {}
        

        for i in range(len(self.filters)):
            # Filters the dataset based on instances of the root node
            
            self.df_b = self.df[self.df[self.root]==self.filters[i]]
            
            for col in self.features:
                if col != self.res:
                    
                    # Creates a new init_dict and sum_dict with filtered data
                    self.init_dict, self.sum_dict = self.built_dictionary(col,self.df_b)
            
            #Child of the root is selected by the lowest gini impurity 
            self.child = self.select_feature()
            
            self.val_child = self.init_dict[self.child]
            
            # If the child feature has only one cluster, positions it to final dictionary
            if self.check_length(self.val_child):
                    
                self.final_dict.append({self.root: self.filters[i],self.child:
                                        [list(self.val_child.keys()),
                                        [list(self.val_child[key].keys())[0] for key in self.val_child.keys()]]})
                    
                
            else:
                # If the child has still multiple clusters, it goes down one level 
                # Positions the result if the second child has only one clusters, skips otherwise
                
                for children in self.val_child.keys():
                    self.df_b2 = self.df_b[self.df_b[self.child]==children]
                    
                    col = [x for x in self.features if x != self.root and x != self.child and x != self.res][0]
                    
                    self.init_dict.update({col: {k: f.groupby(self.res).size().to_dict()
                            for k, f in self.df_b2[[col,self.res]].groupby(col)}})
                    
                    self.val_child2 = self.init_dict[col]
                    
                    if self.check_length(self.val_child2):
                        
                        self.final_dict.append({self.root: self.filters[i],self.child: children, col:
                                        [list(self.val_child2.keys()),
                                        [list(self.val_child2[key].keys())[0] for key in self.val_child2.keys()]]})
                        
        return self.final_dict
    
    
    def predict(self, final_dict, X_test):
        """ Predicts the results given a test set and a tree implementation with built_tree function. """

        self.final_dict = final_dict
        
        self.X_test = X_test
        
        self.pred = []
        
        for row in range(self.X_test.shape[0]):
            res = X_test.iloc[row][self.root]

            self.filtered= []
            
            self.sec_filter = None
            
            for i in range(len(self.final_dict)):
                if self.final_dict[i][self.root]==res:
                    self.filtered.append(self.final_dict[i])
                    
            
            if len(self.filtered) == 1:
                self.selected_dict = self.filtered[0]
            else:
                self.sec_filter = list(self.filtered[0].keys())[1]
                res2 = X_test.iloc[row][self.sec_filter]
                for i in range(len(self.filtered)):
                    if self.filtered[i][self.sec_filter]==res2:
                        self.selected_dict = self.filtered[i]
            
            if self.sec_filter == None:
                self.criteria = [cols for cols in list(self.selected_dict.keys()) if cols != self.root]
                
            else:
                self.criteria = [cols for cols in list(self.selected_dict.keys()) if cols != self.root 
                             and cols != self.sec_filter]

            for cri in self.criteria:
                dict_res = self.selected_dict[cri]
                res = X_test.iloc[row][cri]

                index = dict_res[0].index(res)
                
                self.pred.append(dict_res[1][index])
                
        result_frame = pd.DataFrame(np.array(self.pred).T)
        result_frame.columns =['predicted']
        return result_frame
            
            
    

In [2]:
import pandas as pd
import numpy as np

df_weather= pd.read_csv("weather-data.csv")
df_weather

,Day,Outlook,Temperature,Humidity,Wind,Decision
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rainfall,Mild,High,Weak,Yes
4,5,Rainfall,Cool,Normal,Weak,Yes
5,6,Rainfall,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rainfall,Mild,Normal,Weak,Yes


In [3]:
dt_weather = DecisionTreeClassifier(df_weather)
root_weather = dt_weather.select_feature()
print(f"Root node to start with is {root_weather}.")
print()
decision_tree_weather = dt_weather.build_tree(root_weather)
print('Decision tree for the algorithm:')
print(decision_tree_weather)
print()
pred_w = dt_weather.predict(decision_tree_weather, df_weather)

# Comparing the results 

pd.merge(df_weather, pred_w, left_index=True, right_index=True)

Root node to start with is Outlook.

Decision tree for the algorithm:
[{'Outlook': 'Overcast', 'Temperature': [['Cool', 'Hot', 'Mild'], ['Yes', 'Yes', 'Yes']]}, {'Outlook': 'Rainfall', 'Wind': [['Strong', 'Weak'], ['No', 'Yes']]}, {'Outlook': 'Sunny', 'Humidity': [['High', 'Normal'], ['No', 'Yes']]}]



,Day,Outlook,Temperature,Humidity,Wind,Decision,predicted
0,1,Sunny,Hot,High,Weak,No,No
1,2,Sunny,Hot,High,Strong,No,No
2,3,Overcast,Hot,High,Weak,Yes,Yes
3,4,Rainfall,Mild,High,Weak,Yes,Yes
4,5,Rainfall,Cool,Normal,Weak,Yes,Yes
5,6,Rainfall,Cool,Normal,Strong,No,No
6,7,Overcast,Cool,Normal,Strong,Yes,Yes
7,8,Sunny,Mild,High,Weak,No,No
8,9,Sunny,Cool,Normal,Weak,Yes,Yes
9,10,Rainfall,Mild,Normal,Weak,Yes,Yes


In [4]:
df_titanic = pd.read_excel("titanic_train.xlsx")
df_titanic

,Index,Pclass,Sex,Age,Survived
0,0,1,female,Children,Yes
1,1,1,female,Youth,Yes
2,2,1,female,Adult,Yes
3,3,1,female,Senior,No
4,4,1,male,Children,Yes
5,5,1,male,Youth,No
6,6,1,male,Adult,No
7,7,1,male,Senior,No
8,8,2,female,Children,Yes
9,9,2,female,Youth,Yes


In [5]:
dt_titanic = DecisionTreeClassifier(df_titanic)
root_titanic = dt_titanic.select_feature()
print(f"Root node to start with is {root_titanic}.")
print()

decision_tree_titanic = dt_titanic.build_tree(root_titanic)
print('Decision tree for the algorithm:')
print(decision_tree_titanic)
print()

pred_t = dt_titanic.predict(decision_tree_titanic, df_titanic)

# Comparing the results 

pd.merge(df_titanic, pred_t, left_index=True, right_index=True)

Root node to start with is Age.

Decision tree for the algorithm:
[{'Age': 'Adult', 'Sex': 'female', 'Pclass': [[1, 2, 3], ['Yes', 'Yes', 'No']]}, {'Age': 'Adult', 'Sex': 'male', 'Pclass': [[1, 2, 3], ['No', 'No', 'No']]}, {'Age': 'Children', 'Pclass': 1, 'Sex': [['female', 'male'], ['Yes', 'Yes']]}, {'Age': 'Children', 'Pclass': 2, 'Sex': [['female', 'male'], ['Yes', 'Yes']]}, {'Age': 'Children', 'Pclass': 3, 'Sex': [['female', 'male'], ['Yes', 'No']]}, {'Age': 'Senior', 'Pclass': [[1, 2, 3], ['No', 'No', 'No']]}, {'Age': 'Youth', 'Sex': [['female', 'male'], ['Yes', 'No']]}]



,Index,Pclass,Sex,Age,Survived,predicted
0,0,1,female,Children,Yes,Yes
1,1,1,female,Youth,Yes,Yes
2,2,1,female,Adult,Yes,Yes
3,3,1,female,Senior,No,No
4,4,1,male,Children,Yes,Yes
5,5,1,male,Youth,No,No
6,6,1,male,Adult,No,No
7,7,1,male,Senior,No,No
8,8,2,female,Children,Yes,Yes
9,9,2,female,Youth,Yes,Yes


### Testing the implementation

Since decision tree is built by simplifying the dataset, the maximum accuracy is % 80.92.


In [6]:
df_titanic_test = pd.read_csv("titanic_test.csv")
df_titanic_test = df_titanic_test[['Pclass', 'Sex', 'Age','Survived']]

df_titanic_test['Age'] = pd.cut(df_titanic_test.Age,
                 [0, 14, 24, 64,  np.inf],
                 labels=['Children','Youth','Adult','Senior'])

df_titanic_test['Survived'] = np.where(df_titanic_test['Survived'] == 0 , 
                                   'No', 'Yes')
df_titanic_test


,Pclass,Sex,Age,Survived
0,3,male,Youth,No
1,1,female,Adult,Yes
2,3,female,Adult,Yes
3,1,female,Adult,Yes
4,3,male,Adult,No
...,...,...,...,...
886,2,male,Adult,No
887,1,female,Youth,Yes
888,3,female,Youth,No
889,1,male,Adult,Yes


In [7]:
pred_test = dt_titanic.predict(decision_tree_titanic, df_titanic_test)

merged = pd.merge(df_titanic_test, pred_test, left_index=True, right_index=True)

total_count = df_titanic_test.shape[0]

score = 0
for i in range(total_count):
    if merged.iloc[i]['Survived'] == merged.iloc[i]['predicted']:
        score += 1
        
print(f" Accuracy on the dataset {round((score/total_count)*100, 2)} percent." )

merged


 Accuracy on the dataset 80.92 percent.


,Pclass,Sex,Age,Survived,predicted
0,3,male,Youth,No,No
1,1,female,Adult,Yes,Yes
2,3,female,Adult,Yes,No
3,1,female,Adult,Yes,Yes
4,3,male,Adult,No,No
...,...,...,...,...,...
886,2,male,Adult,No,No
887,1,female,Youth,Yes,Yes
888,3,female,Youth,No,Yes
889,1,male,Adult,Yes,No
